In [ ]:
from pathlib import Path
from models import RandomModel
import numpy as np
import pandas as pd

from utils import root_path, submit
%matplotlib inline


## Data description
The data for this competition includes questions and answers from various StackExchange properties. Your task is to predict target values of 30 labels for each question-answer pair.

The list of 30 target labels are the same as the column names in the `sample_submission.csv` file. Target labels with the prefix `question_` relate to the `question_title` and/or `question_body` features in the data. Target labels with the prefix `answer_` relate to the `answer` feature.

Each row contains a single question and a single answer to that question, along with additional features. The training data contains rows with some duplicated questions (but with different answers). The test data does not contain any duplicated questions.

This is not a binary prediction challenge. Target labels are aggregated from multiple raters, and can have continuous values in the range [0,1]. Therefore, predictions must also be in that range.

Since this is a synchronous re-run competition, you only have access to the Public test set. For planning purposes, the re-run test set is no larger than 10,000 rows, and less than 8 Mb uncompressed.

Additional information about the labels and collection method will be provided by the competition sponsor in the forum.


In [ ]:
current_path = %pwd
current_path = Path(current_path)
competition = current_path.name
competition

In [ ]:
sample_submission = pd.read_csv(root_path / 'data' / competition / 'sample_submission.csv')
sample_submission

In [ ]:
required_output_cols = sample_submission.columns

In [ ]:
test = pd.read_csv(root_path / 'data' / competition / 'test.csv')
test

Apply model, get the result, and save to a csv file (e.g. submit.csv)

In [ ]:
fancy_model = RandomModel
if RandomModel.model is None:
    RandomModel.train(np.array([]), np.array([]))
y_pred = RandomModel.predict(X_test)



Save to csv file

In [ ]:
pd.DataFrame(y_pred).to_csv(current_path / 'submit.csv', index=False)

Submit to kaggle

In [ ]:
submit(competition)

